# Evaluating Vector Retrieval

## Prepare documents with ids

In [2]:
import json

with open('documents-with-ids.json', 'rt') as f_in:
    documents = json.load(f_in)

## Create vector embeddings using a pre-trained model

In [3]:
from sentence_transformers import SentenceTransformer

/Users/patrickcmd/Projects/personal/llm-zoomcamp/env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
model_name = 'multi-qa-MiniLM-L6-cos-v1'
model = SentenceTransformer(model_name)

In [5]:
v = model.encode('I just discovered the course. Can I still join?')

In [6]:
len(v)

384

## Create Vector Mappings and Index in ElasticSearch

In [7]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "text_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions-vector"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions-vector'})

### Adding documents with vector embeddings to elasticsearch index

In [8]:
from tqdm.auto import tqdm

In [9]:
for doc in tqdm(documents):
    question = doc['question']
    text = doc['text']
    qt = question + ' ' + text

    doc['question_vector'] = model.encode(question)
    doc['text_vector'] = model.encode(text)
    doc['question_text_vector'] = model.encode(qt)

100%|██████████| 948/948 [01:02<00:00, 15.07it/s]


In [10]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:08<00:00, 113.50it/s]


## Create ElasticSearch query and vector search query with LLM (Pretrained Model)

In [11]:
query = 'I just discovered the course. Can I still join it?'

In [12]:
v_q = model.encode(query)

In [13]:
def elastic_search_knn(field, vector, course):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

## Groundtruth dataset

In [16]:
import os
import pandas as pd

In [17]:
df_ground_truth = pd.read_csv('ground-truth-data.csv')

In [18]:
# Dropping vague null values
df_ground_truth.dropna(inplace=True)
df_ground_truth.shape

(4207, 3)

In [19]:
df_ground_truth.head()

,question,course,document
0,Can you tell me when the course is starting?,data-engineering-zoomcamp,c02e79ef
1,What is the exact day and hour of the course?,data-engineering-zoomcamp,c02e79ef
2,Is there a specific calendar I should subscrib...,data-engineering-zoomcamp,c02e79ef
3,Can you guide me on how to register for the co...,data-engineering-zoomcamp,c02e79ef
4,What platforms can I use to stay updated on co...,data-engineering-zoomcamp,c02e79ef


In [20]:
ground_truth = df_ground_truth.to_dict(orient='records')

In [21]:
len(ground_truth)

4207

In [22]:
ground_truth[0]

{'question': 'Can you tell me when the course is starting?',
 'course': 'data-engineering-zoomcamp',
 'document': 'c02e79ef'}

### Evaluation metrics functions

In [23]:
# hit rate (recall)
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [24]:
# mrr
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [25]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

### Evaluations with question vector

In [14]:
def question_vector_knn(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_knn('question_vector', v_q, course)

In [26]:
evaluate(ground_truth, question_vector_knn)

100%|██████████| 4207/4207 [01:33<00:00, 45.11it/s]


{'hit_rate': 0.7442357974803898, 'mrr': 0.6316417082640047}

### Evaluations with text vector

In [27]:
def text_vector_knn(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_knn('text_vector', v_q, course)

In [28]:
evaluate(ground_truth, text_vector_knn)

100%|██████████| 4207/4207 [01:21<00:00, 51.56it/s]


{'hit_rate': 0.8012835749940576, 'mrr': 0.6716385389430323}

### Evaluations with question text vector

In [29]:
def question_text_vector_knn(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_knn('question_text_vector', v_q, course)

In [30]:
evaluate(ground_truth, question_text_vector_knn)

100%|██████████| 4207/4207 [01:21<00:00, 51.43it/s]


{'hit_rate': 0.8975516995483718, 'mrr': 0.7871840583155062}

### Evaluations with all vectors combined together at once using single ES query (A bit more completed)

In [31]:
def elastic_search_knn_combined(vector, course):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": [
                    {
                        "script_score": {
                            "query": {
                                "term": {
                                    "course": course
                                }
                            },
                            "script": {
                                "source": """
                                    cosineSimilarity(params.query_vector, 'question_vector') + 
                                    cosineSimilarity(params.query_vector, 'text_vector') + 
                                    cosineSimilarity(params.query_vector, 'question_text_vector') + 
                                    1
                                """,
                                "params": {
                                    "query_vector": vector
                                }
                            }
                        }
                    }
                ],
                "filter": {
                    "term": {
                        "course": course
                    }
                }
            }
        },
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [32]:
def vector_combined_knn(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_knn_combined(v_q, course)

In [33]:
evaluate(ground_truth, vector_combined_knn)

100%|██████████| 4207/4207 [01:39<00:00, 42.11it/s]


{'hit_rate': 0.8790111718564297, 'mrr': 0.7697844861738382}